In [1]:
import pickle, random, copy, platform, os, time
import numpy as np
import tensorflow as tf
import email_sending
import tflearn
import tflearn.activations as activations
from PIL import Image, ImageDraw
from __future__ import division, print_function, absolute_import
from tflearn.layers.core import input_data, dropout, fully_connected, flatten, reshape
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d,avg_pool_2d, highway_conv_2d,global_avg_pool
from tflearn.layers.normalization import local_response_normalization, batch_normalization
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.utils import repeat

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
def read_data():
    with open('resnet_data', 'rb') as f:
        data = pickle.load(f)
        Xtr = data[0]
        Xtr = np.reshape(Xtr, (-1,75,75,2))
        Ytr = data[1]
        Xte =data[2]
        Xte = np.reshape(Xte, (-1,75,75,2))
        rf =data[3]
    return Xtr, Ytr, Xte, rf

# expresses the label data in one-hot encoding.
def onehot_encoding (Y, y_class):
    Y_onehot = np.zeros((Y.size, int(y_class)))
    for i in range(Y.size):
        Y_onehot[i][Y[i]] = 1
    return Y_onehot

In [3]:
def convnet(x, classes):
    network = conv_2d(x, 32, 5, activation='elu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = batch_normalization(network) #try with batch norm
    network = conv_2d(network, 64, 5, activation='elu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = batch_normalization(network)
    network = conv_2d(network, 128, 5, activation='elu', regularizer="L2")
    network = batch_normalization(network)
    network = fully_connected(network, 512, activation='elu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 2048, activation='elu')
    network = dropout(network, 0.5)
    network = fully_connected(network, classes, activation='softmax')
    return network

def resnet(x, classes):
    # Building Residual Network
    net = tflearn.conv_2d(x, 64, 3, activation='elu', bias=False)
    # Residual blocks
    net = tflearn.residual_bottleneck(net, 3, 16, 64)
    net = tflearn.residual_bottleneck(net, 1, 32, 128, downsample=True)
    net = tflearn.residual_bottleneck(net, 2, 32, 128)
    net = tflearn.residual_bottleneck(net, 1, 64, 256, downsample=True)
    net = tflearn.residual_bottleneck(net, 2, 64, 256)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'elu')
    net = tflearn.global_avg_pool(net)
    # Regression
    net = tflearn.fully_connected(net, classes, activation='softmax')
    return net

In [4]:
def resnet32(x, classes, n = 5): #classes =2
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.residual_block(net, n, 16)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    net = tflearn.residual_block(net, n - 1, 32)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    net = tflearn.residual_block(net, n - 1, 64)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, classes, activation='softmax')

    return net

In [5]:
def resnext(x, classes, n =5):
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.resnext_block(net, n, 16, 32)
    net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
    net = tflearn.resnext_block(net, n-1, 32, 32)
    net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
    net = tflearn.resnext_block(net, n-1, 64, 32)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, classes, activation='softmax')
    
    return net

In [6]:
def vgg(x, classes):    
    network = conv_2d(x, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 128, 3, activation='relu')
    network = conv_2d(network, 128, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, classes, activation='softmax')
    
    return network

In [7]:
def alexnet(x, classes):  
    network = conv_2d(x, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 2, activation='softmax')
    
    return network

In [8]:
def conv_high(x, classes):
    for i in range(3):
        for j in [3, 2, 1]: 
            network = highway_conv_2d(network, 16, j, activation='elu')
        network = max_pool_2d(network, 2)
        network = batch_normalization(network)

    network = fully_connected(network, 128, activation='elu')
    network = fully_connected(network, 256, activation='elu')
    network = fully_connected(network, 2, activation='softmax')
    return network

In [9]:
def densenet(x, classes):   
    # Building Residual Network
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.densenet_block(net, nb_layers, k)
    net = tflearn.densenet_block(net, nb_layers, k)
    net = tflearn.densenet_block(net, nb_layers, k)
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    return net

In [10]:
def block35(net, scale=1.0, activation="relu"):
    tower_conv = relu(batch_normalization(conv_2d(net, 32, 1, bias=False, activation=None, name='Conv2d_1x1')))
    tower_conv1_0 = relu(batch_normalization(conv_2d(net, 32, 1, bias=False, activation=None,name='Conv2d_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 32, 3, bias=False, activation=None,name='Conv2d_0b_3x3')))
    tower_conv2_0 = relu(batch_normalization(conv_2d(net, 32, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv2_1 = relu(batch_normalization(conv_2d(tower_conv2_0, 48,3, bias=False, activation=None, name='Conv2d_0b_3x3')))
    tower_conv2_2 = relu(batch_normalization(conv_2d(tower_conv2_1, 64,3, bias=False, activation=None, name='Conv2d_0c_3x3')))
    tower_mixed = merge([tower_conv, tower_conv1_1, tower_conv2_2], mode='concat', axis=3)
    tower_out = relu(batch_normalization(conv_2d(tower_mixed, net.get_shape()[3], 1, bias=False, activation=None, name='Conv2d_1x1')))
    net += scale * tower_out
    if activation:
        if isinstance(activation, str):
            net = activations.get(activation)(net)
        elif hasattr(activation, '__call__'):
            net = activation(net)
        else:
            raise ValueError("Invalid Activation.")
    return net

def block17(net, scale=1.0, activation="relu"):
    tower_conv = relu(batch_normalization(conv_2d(net, 192, 1, bias=False, activation=None, name='Conv2d_1x1')))
    tower_conv_1_0 = relu(batch_normalization(conv_2d(net, 128, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv_1_1 = relu(batch_normalization(conv_2d(tower_conv_1_0, 160,[1,7], bias=False, activation=None,name='Conv2d_0b_1x7')))
    tower_conv_1_2 = relu(batch_normalization(conv_2d(tower_conv_1_1, 192, [7,1], bias=False, activation=None,name='Conv2d_0c_7x1')))
    tower_mixed = merge([tower_conv,tower_conv_1_2], mode='concat', axis=3)
    tower_out = relu(batch_normalization(conv_2d(tower_mixed, net.get_shape()[3], 1, bias=False, activation=None, name='Conv2d_1x1')))
    net += scale * tower_out
    if activation:
        if isinstance(activation, str):
            net = activations.get(activation)(net)
        elif hasattr(activation, '__call__'):
            net = activation(net)
        else:
            raise ValueError("Invalid Activation.")
    return net


def block8(net, scale=1.0, activation="relu"):
    tower_conv = relu(batch_normalization(conv_2d(net, 192, 1, bias=False, activation=None, name='Conv2d_1x1')))
    tower_conv1_0 = relu(batch_normalization(conv_2d(net, 192, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 224, [1,3], bias=False, activation=None, name='Conv2d_0b_1x3')))
    tower_conv1_2 = relu(batch_normalization(conv_2d(tower_conv1_1, 256, [3,1], bias=False, name='Conv2d_0c_3x1')))
    tower_mixed = merge([tower_conv,tower_conv1_2], mode='concat', axis=3)
    tower_out = relu(batch_normalization(conv_2d(tower_mixed, net.get_shape()[3], 1, bias=False, activation=None, name='Conv2d_1x1')))
    net += scale * tower_out
    if activation:
        if isinstance(activation, str):
            net = activations.get(activation)(net)
        elif hasattr(activation, '__call__'):
            net = activation(net)
        else:
            raise ValueError("Invalid Activation.")
    return net

In [11]:
def inception_resnet_v2(x, classes):
    dropout_keep_prob = 0.8

    conv1a_3_3 = relu(batch_normalization(conv_2d(x, 32, 3, strides=2, bias=False, padding='VALID',activation=None,name='Conv2d_1a_3x3')))
    conv2a_3_3 = relu(batch_normalization(conv_2d(conv1a_3_3, 32, 3, bias=False, padding='VALID',activation=None, name='Conv2d_2a_3x3')))
    conv2b_3_3 = relu(batch_normalization(conv_2d(conv2a_3_3, 64, 3, bias=False, activation=None, name='Conv2d_2b_3x3')))
    maxpool3a_3_3 = max_pool_2d(conv2b_3_3, 3, strides=2, padding='VALID', name='MaxPool_3a_3x3')
    conv3b_1_1 = relu(batch_normalization(conv_2d(maxpool3a_3_3, 80, 1, bias=False, padding='VALID',activation=None, name='Conv2d_3b_1x1')))
    conv4a_3_3 = relu(batch_normalization(conv_2d(conv3b_1_1, 192, 3, bias=False, padding='VALID',activation=None, name='Conv2d_4a_3x3')))
    maxpool5a_3_3 = max_pool_2d(conv4a_3_3, 3, strides=2, padding='VALID', name='MaxPool_5a_3x3')

    tower_conv = relu(batch_normalization(conv_2d(maxpool5a_3_3, 96, 1, bias=False, activation=None, name='Conv2d_5b_b0_1x1')))

    tower_conv1_0 = relu(batch_normalization(conv_2d(maxpool5a_3_3, 48, 1, bias=False, activation=None, name='Conv2d_5b_b1_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 64, 5, bias=False, activation=None, name='Conv2d_5b_b1_0b_5x5')))

    tower_conv2_0 = relu(batch_normalization(conv_2d(maxpool5a_3_3, 64, 1, bias=False, activation=None, name='Conv2d_5b_b2_0a_1x1')))
    tower_conv2_1 = relu(batch_normalization(conv_2d(tower_conv2_0, 96, 3, bias=False, activation=None, name='Conv2d_5b_b2_0b_3x3')))
    tower_conv2_2 = relu(batch_normalization(conv_2d(tower_conv2_1, 96, 3, bias=False, activation=None,name='Conv2d_5b_b2_0c_3x3')))

    tower_pool3_0 = avg_pool_2d(maxpool5a_3_3, 3, strides=1, padding='same', name='AvgPool_5b_b3_0a_3x3')
    tower_conv3_1 = relu(batch_normalization(conv_2d(tower_pool3_0, 64, 1, bias=False, activation=None,name='Conv2d_5b_b3_0b_1x1')))

    tower_5b_out = merge([tower_conv, tower_conv1_1, tower_conv2_2, tower_conv3_1], mode='concat', axis=3)

    net = repeat(tower_5b_out, 10, block35, scale=0.17)

    tower_conv = relu(batch_normalization(conv_2d(net, 384, 3, bias=False, strides=2,activation=None, padding='VALID', name='Conv2d_6a_b0_0a_3x3')))
    tower_conv1_0 = relu(batch_normalization(conv_2d(net, 256, 1, bias=False, activation=None, name='Conv2d_6a_b1_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 256, 3, bias=False, activation=None, name='Conv2d_6a_b1_0b_3x3')))
    tower_conv1_2 = relu(batch_normalization(conv_2d(tower_conv1_1, 384, 3, bias=False, strides=2, padding='VALID', activation=None,name='Conv2d_6a_b1_0c_3x3')))
    tower_pool = max_pool_2d(net, 3, strides=2, padding='VALID',name='MaxPool_1a_3x3')
    net = merge([tower_conv, tower_conv1_2, tower_pool], mode='concat', axis=3)
    net = repeat(net, 20, block17, scale=0.1)

    tower_conv = relu(batch_normalization(conv_2d(net, 256, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv0_1 = relu(batch_normalization(conv_2d(tower_conv, 384, 3, bias=False, strides=2, padding='VALID', activation=None,name='Conv2d_0a_1x1')))

    tower_conv1 = relu(batch_normalization(conv_2d(net, 256, 1, bias=False, padding='VALID', activation=None,name='Conv2d_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1,288,3, bias=False, strides=2, padding='VALID',activation=None, name='COnv2d_1a_3x3')))

    tower_conv2 = relu(batch_normalization(conv_2d(net, 256,1, bias=False, activation=None,name='Conv2d_0a_1x1')))
    tower_conv2_1 = relu(batch_normalization(conv_2d(tower_conv2, 288,3, bias=False, name='Conv2d_0b_3x3',activation=None)))
    tower_conv2_2 = relu(batch_normalization(conv_2d(tower_conv2_1, 320, 3, bias=False, strides=2, padding='VALID',activation=None, name='Conv2d_1a_3x3')))

    tower_pool = max_pool_2d(net, 3, strides=2, padding='VALID', name='MaxPool_1a_3x3')
    net = merge([tower_conv0_1, tower_conv1_1,tower_conv2_2, tower_pool], mode='concat', axis=3)

    net = repeat(net, 9, block8, scale=0.2)
    net = block8(net, activation=None)

    net = relu(batch_normalization(conv_2d(net, 1536, 1, bias=False, activation=None, name='Conv2d_7b_1x1')))
    net = avg_pool_2d(net, net.get_shape().as_list()[1:3],strides=2, padding='VALID', name='AvgPool_1a_8x8')
    net = flatten(net)
    net = dropout(net, dropout_keep_prob)
    loss = fully_connected(net, classes,activation='softmax')
    
    return loss

In [12]:
def googlenet(x, classes):
    conv1_7_7 = conv_2d(x, 64, 7, strides=2, activation='relu', name='conv1_7_7_s2')
    pool1_3_3 = max_pool_2d(conv1_7_7, 3, strides=2)
    pool1_3_3 = local_response_normalization(pool1_3_3)
    conv2_3_3_reduce = conv_2d(pool1_3_3, 64, 1, activation='relu', name='conv2_3_3_reduce')
    conv2_3_3 = conv_2d(conv2_3_3_reduce, 192, 3, activation='relu', name='conv2_3_3')
    conv2_3_3 = local_response_normalization(conv2_3_3)
    pool2_3_3 = max_pool_2d(conv2_3_3, kernel_size=3, strides=2, name='pool2_3_3_s2')

    # 3a
    inception_3a_1_1 = conv_2d(pool2_3_3, 64, 1, activation='relu', name='inception_3a_1_1')
    inception_3a_3_3_reduce = conv_2d(pool2_3_3, 96, 1, activation='relu', name='inception_3a_3_3_reduce')
    inception_3a_3_3 = conv_2d(inception_3a_3_3_reduce, 128, filter_size=3,  activation='relu', name='inception_3a_3_3')
    inception_3a_5_5_reduce = conv_2d(pool2_3_3, 16, filter_size=1, activation='relu', name='inception_3a_5_5_reduce')
    inception_3a_5_5 = conv_2d(inception_3a_5_5_reduce, 32, filter_size=5, activation='relu', name='inception_3a_5_5')
    inception_3a_pool = max_pool_2d(pool2_3_3, kernel_size=3, strides=1, name='inception_3a_pool')
    inception_3a_pool_1_1 = conv_2d(inception_3a_pool, 32, filter_size=1, activation='relu', name='inception_3a_pool_1_1')
    inception_3a_output = merge([inception_3a_1_1, inception_3a_3_3, inception_3a_5_5, inception_3a_pool_1_1], mode='concat', axis=3)

    # 3b
    inception_3b_1_1 = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_1_1')
    inception_3b_3_3_reduce = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_3_3_reduce')
    inception_3b_3_3 = conv_2d(inception_3b_3_3_reduce, 192, filter_size=3, activation='relu', name='inception_3b_3_3')
    inception_3b_5_5_reduce = conv_2d(inception_3a_output, 32, filter_size=1, activation='relu', name='inception_3b_5_5_reduce')
    inception_3b_5_5 = conv_2d(inception_3b_5_5_reduce, 96, filter_size=5,  name='inception_3b_5_5')
    inception_3b_pool = max_pool_2d(inception_3a_output, kernel_size=3, strides=1,  name='inception_3b_pool')
    inception_3b_pool_1_1 = conv_2d(inception_3b_pool, 64, filter_size=1, activation='relu', name='inception_3b_pool_1_1')
    inception_3b_output = merge([inception_3b_1_1, inception_3b_3_3, inception_3b_5_5, inception_3b_pool_1_1], mode='concat', axis=3, name='inception_3b_output')
    pool3_3_3 = max_pool_2d(inception_3b_output, kernel_size=3, strides=2, name='pool3_3_3')

    # 4a
    inception_4a_1_1 = conv_2d(pool3_3_3, 192, filter_size=1, activation='relu', name='inception_4a_1_1')
    inception_4a_3_3_reduce = conv_2d(pool3_3_3, 96, filter_size=1, activation='relu', name='inception_4a_3_3_reduce')
    inception_4a_3_3 = conv_2d(inception_4a_3_3_reduce, 208, filter_size=3,  activation='relu', name='inception_4a_3_3')
    inception_4a_5_5_reduce = conv_2d(pool3_3_3, 16, filter_size=1, activation='relu', name='inception_4a_5_5_reduce')
    inception_4a_5_5 = conv_2d(inception_4a_5_5_reduce, 48, filter_size=5,  activation='relu', name='inception_4a_5_5')
    inception_4a_pool = max_pool_2d(pool3_3_3, kernel_size=3, strides=1,  name='inception_4a_pool')
    inception_4a_pool_1_1 = conv_2d(inception_4a_pool, 64, filter_size=1, activation='relu', name='inception_4a_pool_1_1')
    inception_4a_output = merge([inception_4a_1_1, inception_4a_3_3, inception_4a_5_5, inception_4a_pool_1_1], mode='concat', axis=3, name='inception_4a_output')

    # 4b
    inception_4b_1_1 = conv_2d(inception_4a_output, 160, filter_size=1, activation='relu', name='inception_4a_1_1')
    inception_4b_3_3_reduce = conv_2d(inception_4a_output, 112, filter_size=1, activation='relu', name='inception_4b_3_3_reduce')
    inception_4b_3_3 = conv_2d(inception_4b_3_3_reduce, 224, filter_size=3, activation='relu', name='inception_4b_3_3')
    inception_4b_5_5_reduce = conv_2d(inception_4a_output, 24, filter_size=1, activation='relu', name='inception_4b_5_5_reduce')
    inception_4b_5_5 = conv_2d(inception_4b_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4b_5_5')
    inception_4b_pool = max_pool_2d(inception_4a_output, kernel_size=3, strides=1,  name='inception_4b_pool')
    inception_4b_pool_1_1 = conv_2d(inception_4b_pool, 64, filter_size=1, activation='relu', name='inception_4b_pool_1_1')
    inception_4b_output = merge([inception_4b_1_1, inception_4b_3_3, inception_4b_5_5, inception_4b_pool_1_1], mode='concat', axis=3, name='inception_4b_output')

    # 4c
    inception_4c_1_1 = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_1_1')
    inception_4c_3_3_reduce = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_3_3_reduce')
    inception_4c_3_3 = conv_2d(inception_4c_3_3_reduce, 256,  filter_size=3, activation='relu', name='inception_4c_3_3')
    inception_4c_5_5_reduce = conv_2d(inception_4b_output, 24, filter_size=1, activation='relu', name='inception_4c_5_5_reduce')
    inception_4c_5_5 = conv_2d(inception_4c_5_5_reduce, 64,  filter_size=5, activation='relu', name='inception_4c_5_5')
    inception_4c_pool = max_pool_2d(inception_4b_output, kernel_size=3, strides=1)
    inception_4c_pool_1_1 = conv_2d(inception_4c_pool, 64, filter_size=1, activation='relu', name='inception_4c_pool_1_1')
    inception_4c_output = merge([inception_4c_1_1, inception_4c_3_3, inception_4c_5_5, inception_4c_pool_1_1], mode='concat', axis=3, name='inception_4c_output')

    # 4d
    inception_4d_1_1 = conv_2d(inception_4c_output, 112, filter_size=1, activation='relu', name='inception_4d_1_1')
    inception_4d_3_3_reduce = conv_2d(inception_4c_output, 144, filter_size=1, activation='relu', name='inception_4d_3_3_reduce')
    inception_4d_3_3 = conv_2d(inception_4d_3_3_reduce, 288, filter_size=3, activation='relu', name='inception_4d_3_3')
    inception_4d_5_5_reduce = conv_2d(inception_4c_output, 32, filter_size=1, activation='relu', name='inception_4d_5_5_reduce')
    inception_4d_5_5 = conv_2d(inception_4d_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4d_5_5')
    inception_4d_pool = max_pool_2d(inception_4c_output, kernel_size=3, strides=1,  name='inception_4d_pool')
    inception_4d_pool_1_1 = conv_2d(inception_4d_pool, 64, filter_size=1, activation='relu', name='inception_4d_pool_1_1')
    inception_4d_output = merge([inception_4d_1_1, inception_4d_3_3, inception_4d_5_5, inception_4d_pool_1_1], mode='concat', axis=3, name='inception_4d_output')

    # 4e
    inception_4e_1_1 = conv_2d(inception_4d_output, 256, filter_size=1, activation='relu', name='inception_4e_1_1')
    inception_4e_3_3_reduce = conv_2d(inception_4d_output, 160, filter_size=1, activation='relu', name='inception_4e_3_3_reduce')
    inception_4e_3_3 = conv_2d(inception_4e_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_4e_3_3')
    inception_4e_5_5_reduce = conv_2d(inception_4d_output, 32, filter_size=1, activation='relu', name='inception_4e_5_5_reduce')
    inception_4e_5_5 = conv_2d(inception_4e_5_5_reduce, 128,  filter_size=5, activation='relu', name='inception_4e_5_5')
    inception_4e_pool = max_pool_2d(inception_4d_output, kernel_size=3, strides=1,  name='inception_4e_pool')
    inception_4e_pool_1_1 = conv_2d(inception_4e_pool, 128, filter_size=1, activation='relu', name='inception_4e_pool_1_1')
    inception_4e_output = merge([inception_4e_1_1, inception_4e_3_3, inception_4e_5_5, inception_4e_pool_1_1], axis=3, mode='concat')
    pool4_3_3 = max_pool_2d(inception_4e_output, kernel_size=3, strides=2, name='pool_3_3')

    # 5a
    inception_5a_1_1 = conv_2d(pool4_3_3, 256, filter_size=1, activation='relu', name='inception_5a_1_1')
    inception_5a_3_3_reduce = conv_2d(pool4_3_3, 160, filter_size=1, activation='relu', name='inception_5a_3_3_reduce')
    inception_5a_3_3 = conv_2d(inception_5a_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_5a_3_3')
    inception_5a_5_5_reduce = conv_2d(pool4_3_3, 32, filter_size=1, activation='relu', name='inception_5a_5_5_reduce')
    inception_5a_5_5 = conv_2d(inception_5a_5_5_reduce, 128, filter_size=5,  activation='relu', name='inception_5a_5_5')
    inception_5a_pool = max_pool_2d(pool4_3_3, kernel_size=3, strides=1,  name='inception_5a_pool')
    inception_5a_pool_1_1 = conv_2d(inception_5a_pool, 128, filter_size=1, activation='relu', name='inception_5a_pool_1_1')
    inception_5a_output = merge([inception_5a_1_1, inception_5a_3_3, inception_5a_5_5, inception_5a_pool_1_1], axis=3, mode='concat')

    # 5b
    inception_5b_1_1 = conv_2d(inception_5a_output, 384, filter_size=1, activation='relu', name='inception_5b_1_1')
    inception_5b_3_3_reduce = conv_2d(inception_5a_output, 192, filter_size=1, activation='relu', name='inception_5b_3_3_reduce')
    inception_5b_3_3 = conv_2d(inception_5b_3_3_reduce, 384,  filter_size=3, activation='relu', name='inception_5b_3_3')
    inception_5b_5_5_reduce = conv_2d(inception_5a_output, 48, filter_size=1, activation='relu', name='inception_5b_5_5_reduce')
    inception_5b_5_5 = conv_2d(inception_5b_5_5_reduce, 128, filter_size=5, activation='relu', name='inception_5b_5_5')
    inception_5b_pool = max_pool_2d(inception_5a_output, kernel_size=3, strides=1,  name='inception_5b_pool')
    inception_5b_pool_1_1 = conv_2d(inception_5b_pool, 128, filter_size=1, activation='relu', name='inception_5b_pool_1_1')
    inception_5b_output = merge([inception_5b_1_1, inception_5b_3_3, inception_5b_5_5, inception_5b_pool_1_1], axis=3, mode='concat')
    pool5_7_7 = avg_pool_2d(inception_5b_output, kernel_size=7, strides=1)
    pool5_7_7 = dropout(pool5_7_7, 0.4)

    # fc
    net = fully_connected(pool5_7_7, classes, activation='softmax')
    
    return net

In [13]:
n_ensemble = 7
ensemble = np.zeros((n_ensemble, 8424, 2))

def predict(model, test_data, ref, mode, count):
    #8424=8*81*13
    batch_size =104
    iteration = int(len(test_data)/batch_size)  #iter=81  
    
    with open("C:/Users/User/Desktop/kaggle/result_{0}.txt".format(mode), "w") as f:
        f.write("id,is_iceberg\n")
        for i in range(iteration):
            pred = model.predict(test_data[i*batch_size: (i+1)*batch_size])
            ensemble[count][i*batch_size: (i+1)*batch_size] = pred
            for j in range(batch_size):
                data = "%s,%f\n" % (ref[i*batch_size+j], pred[j][1])
                f.write(data)

In [14]:
save_dir = 'checkpoints/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
def get_save_path(net_name):
    return save_dir + str(net_name)

In [15]:
class EarlyStoppingCallback(tflearn.callbacks.Callback):
    def __init__(self, val_acc_thresh):
        """ Note: We are free to define our init function however we please. """
        self.val_acc_thresh = val_acc_thresh
    
    def on_epoch_end(self, training_state):
        """ """
        # Apparently this can happen.
        if training_state.val_acc is None: return
        if training_state.val_acc > self.val_acc_thresh:
            raise StopIteration

In [16]:
Xtr, Ytr, Xte, ref = read_data()

Xtr, Ytr = tflearn.data_utils.shuffle(Xtr, Ytr)
Ytr = onehot_encoding (Ytr, 2)

Xtr, mean = tflearn.data_utils.featurewise_zero_center(Xtr)


#data split
X, Y = Xtr[:1200], Ytr[:1200]
validX, validY =Xtr[1200:1500], Ytr[1200:1500]
testX, testY = Xtr[1500:],Ytr[1500:]

X= tflearn.data_utils.featurewise_zero_center(X, mean)
testX = tflearn.data_utils.featurewise_zero_center(testX, mean)
validX = tflearn.data_utils.featurewise_zero_center(validX, mean)

#augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_90degrees_rotation(rotations=[0, 1, 2, 3])
img_aug.add_random_crop([75,75,2], padding=4)

#input tensor
Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', name ='input', data_augmentation=img_aug)
early_stopping_cb = EarlyStoppingCallback(val_acc_thresh=0.95) 

In [17]:
def train(mode, num_epochs = 20, batch_size=100, classes=2, n_ens = 0):
    
    if mode =='convnet':
        network = tflearn.regression(convnet(Input,classes),optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True, callbacks=early_stopping_cb,batch_size=batch_size, run_id='convnet')
        predict(model, Xte, ref, mode, n_ens)
        
    elif mode =='resnet':
        network = tflearn.regression(resnet(Input,classes),optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True, callbacks=early_stopping_cb,batch_size=batch_size, run_id='resnet')
        predict(model, Xte, ref, mode, n_ens)
  
    elif mode =='resnet32':
        network = tflearn.regression(resnet32(Input,classes),optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True, callbacks=early_stopping_cb,batch_size=batch_size, run_id='resnet32')
        predict(model, Xte, ref, mode, n_ens)

    elif mode =='inception_resnet_v2':        
        network = tflearn.regression(inception_resnet_v2(Input,classes),
                                     optimizer='adam',loss='categorical_crossentropy',name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, show_metric=True, batch_size=batch_size,
                  validation_set=({'input': validX}, {'target': validY}),callbacks=early_stopping_cb, run_id ='inception_resnet_v2')
        predict(model, Xte, ref, mode, n_ens)

    elif mode=='resnext':
        network = tflearn.regression(resnext(Input,classes), optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True, batch_size=batch_size, callbacks=early_stopping_cb,run_id='resnext')
        predict(model, Xte, ref, mode, n_ens)
        
    elif mode=='googlenet':
        network = regression(googlenet(Input,classes),  optimizer='adam',loss='categorical_crossentropy',name ='target')
        model = tflearn.DNN(network, tensorboard_verbose=3, tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True,  batch_size=batch_size, callbacks=early_stopping_cb, run_id='googlenet')
        predict(model, Xte, ref, mode, n_ens)

    elif mode=='vgg':
        network = regression(vgg(Input,classes), optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3, tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True,  batch_size=batch_size, callbacks=early_stopping_cb, run_id='vgg')
        predict(model, Xte, ref, mode, n_ens)
    
    elif mode =='alexnet':
        network = regression(alexnet(Input,classes), optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3, tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True,  batch_size=batch_size, callbacks=early_stopping_cb, run_id='alexnet')
        predict(model, Xte, ref, mode, n_ens)
                  
    elif mode =='conv_high':
        network = regression(conv_high(Input,classes), optimizer='adam',loss='categorical_crossentropy', name ='target')
        model = tflearn.DNN(network,tensorboard_verbose=3, tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True,  batch_size=batch_size, callbacks=early_stopping_cb, run_id='conv_high')
        predict(model, Xte, ref, mode, n_ens)
        
    elif mode == 'densenet':
        network = regression(densenet(Input,classes), optimizer='adam',loss='categorical_crossentropy', name ='target') 
        model = tflearn.DNN(network,tensorboard_verbose=3, tensorboard_dir='c:\\tflearn_logs\\')
        model.fit({'input': X}, {'target': Y}, n_epoch=num_epochs, validation_set=({'input': validX}, {'target': validY}),
                  show_metric=True,  batch_size=batch_size, callbacks=early_stopping_cb, run_id='densenet')
        predict(model, Xte, ref, mode, n_ens)

    else:
        print("Please Select Mode in ['resnet32', 'inception_resnet_v2', 'resnext', 'googlenet','vgg', 'convnet', 'resnet'\
               'alexnet' 'conv_high', 'densenet']")

    model.save(get_save_path(mode))

    return model

In [18]:
if __name__ == '__main__':
    train('conv_high', num_epochs = 1, batch_size=100, classes=2, n_ens= 0)

UnboundLocalError: local variable 'network' referenced before assignment

train('resnet', num_epochs = 40, batch_size=100, classes=2, n_ens= 0)
train('vgg', num_epochs = 40, batch_size=100, classes=2, n_ens= 1)
train('googlenet', num_epochs = 40, batch_size=100, classes=2, n_ens= 2)
train('alexnet', num_epochs = 40, batch_size=100, classes=2, n_ens= 3)
train('densenet', num_epochs = 40, batch_size=100, classes=2, n_ens= 4)
train('conv_high', num_epochs = 40, batch_size=100, classes=2, n_ens= 5)
train('convnet', num_epochs = 40, batch_size=100, classes=2, n_ens= 6)